# Part-C: Programming Implementation (Python / Qiskit)


## 1.   Write a function that builds a Qiskit oracle circuit for Simon’s function with a secret string s. The oracle acts on two registers (each n qubits): input and output registers. It implements
```markdown
                    Uf(|x⟩|y⟩) = |x⟩|y ⊕f(x)⟩
```
## where f(x)=f(z) ↔x = z ⊕s


In [1]:
import numpy as np
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

In [2]:
def simon_oracle(n=3, s='101'):
    s = s[::-1]
    qc_o = QuantumCircuit(2 * n)

    for i in range(n):
        qc_o.cx(i, n + i)

    if '1' not in s:
        return qc_o
    i = s.find('1')
    for j in range(n):
        if s[j] == '1':
            qc_o.cx(i, (j)+n)
    
    return qc_o

## 2.   Use the oracle from question 1 to implement the quantum side of a single run of Simon’s algorithm(steps 1-5) in qiskit for n qubits and print the measured bitstring.


In [ ]:
def simons_algorithm(n=3, s='101'):
    qc = QuantumCircuit(2 * n, n)

    # Step 1: Hadamard on input register
    for i in range(n):
        qc.h(i)

    # Step 2: Apply oracle
    # Step 3: Measure the Second Register (not sure, why needed?)
    oracle = simon_oracle(n, s)
    qc.compose(oracle, inplace=True)

    # Step 4: Hadamard again on input register
    for i in range(n):
        qc.h(i)

    # Step 5: Measurement of input register
    for i in range(n):
        qc.measure(i, i)

    return qc

In [4]:
n = 3
s = '110'
qc = simons_algorithm(n, s)
# qc.draw('mpl')

# Run the circuit
sim = AerSimulator()
tqc = transpile(qc, sim)
result = sim.run(tqc, shots=1024).result()
counts = result.get_counts()
print("Measurement Results:")
for bitstring, prob in counts.items():
    print(f"{bitstring}: {prob:.3f}")


Measurement Results:
111: 280.000
110: 261.000
000: 248.000
001: 235.000


## 3.  [OPTIONAL] Write code to run simon’s algorithm in full. Note: Algorithmically solving a set of linear equations is not easy, you can try to come up with your own algorithm or look into using the sympy or galois packages. This problem is optional as this is not a linear algebra course!

In [5]:
def solve_linear_mod2(equations: list[str]) -> str:
    n = len(equations[0])
    m = len(equations)
    A = np.array([[int(bit) for bit in eq] for eq in equations], dtype=np.uint8)

    row = 0
    for col in range(n):
        pivot = None
        for r in range(row, m):
            if A[r][col] == 1:
                pivot = r
                break
        if pivot is None:
            continue
        A[[row, pivot]] = A[[pivot, row]]
        for r in range(m):
            if r != row and A[r][col] == 1:
                A[r] ^= A[row]
        row += 1

    # Try non-zero solution for s
    for guess in range(1, 2**n):
        s = np.array([(guess >> i) & 1 for i in reversed(range(n))], dtype=np.uint8)
        if np.all((A @ s) % 2 == 0):
            return ''.join(str(b) for b in s)

    return "No solution found"

In [6]:
equations = [k for k in counts.keys() if k != '0' * len(s)]
print(f"\nEquations from measurements: {equations}")
s_found = solve_linear_mod2(equations)
print(f"\nRecovered hidden string s: {s_found}")


Equations from measurements: ['111', '110', '001']

Recovered hidden string s: 110
